In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

Requirement already up-to-date: sklearn in c:\users\kandr\appdata\local\continuum\anaconda3\lib\site-packages (0.0)


In [2]:
# install joblib. This will be used to save your model. 
!pip install joblib

In [3]:
!pip install tensorflow

In [4]:
# Dependencies
import numpy as np
import pandas as pd

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.metrics import classification_report
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
import tensorflow as tf
#tensorflow.keras.__version__
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [6]:
import warnings
warnings.simplefilter('ignore')

In [7]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [8]:
df.columns

Index(['koi_disposition', 'koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co',
       'koi_fpflag_ec', 'koi_period', 'koi_period_err1', 'koi_period_err2',
       'koi_time0bk', 'koi_time0bk_err1', 'koi_time0bk_err2', 'koi_impact',
       'koi_impact_err1', 'koi_impact_err2', 'koi_duration',
       'koi_duration_err1', 'koi_duration_err2', 'koi_depth', 'koi_depth_err1',
       'koi_depth_err2', 'koi_prad', 'koi_prad_err1', 'koi_prad_err2',
       'koi_teq', 'koi_insol', 'koi_insol_err1', 'koi_insol_err2',
       'koi_model_snr', 'koi_tce_plnt_num', 'koi_steff', 'koi_steff_err1',
       'koi_steff_err2', 'koi_slogg', 'koi_slogg_err1', 'koi_slogg_err2',
       'koi_srad', 'koi_srad_err1', 'koi_srad_err2', 'ra', 'dec',
       'koi_kepmag'],
      dtype='object')

In [9]:
Xtemp = df[['koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co', 'koi_fpflag_ec', 'koi_period','koi_time0bk','koi_slogg','koi_srad','koi_impact','koi_duration','koi_depth','koi_prad','koi_teq','koi_insol','koi_model_snr','koi_steff','koi_slogg','koi_srad','ra','dec','koi_kepmag']]
Xtemp

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_time0bk,koi_slogg,koi_srad,koi_impact,koi_duration,...,koi_prad,koi_teq,koi_insol,koi_model_snr,koi_steff,koi_slogg,koi_srad,ra,dec,koi_kepmag
0,0,0,0,0,54.418383,162.513840,4.467,0.927,0.586,4.50700,...,2.83,443,9.11,25.8,5455,4.467,0.927,291.93423,48.141651,15.347
1,0,1,0,0,19.899140,175.850252,4.544,0.868,0.969,1.78220,...,14.60,638,39.30,76.3,5853,4.544,0.868,297.00482,48.134129,15.436
2,0,1,0,0,1.736952,170.307565,4.564,0.791,1.276,2.40641,...,33.46,1395,891.96,505.6,5805,4.564,0.791,285.53461,48.285210,15.597
3,0,0,0,0,2.525592,171.595550,4.438,1.046,0.701,1.65450,...,2.75,1406,926.16,40.9,6031,4.438,1.046,288.75488,48.226200,15.509
4,0,0,0,0,4.134435,172.979370,4.486,0.972,0.762,3.14020,...,2.77,1160,427.65,40.2,6046,4.486,0.972,296.28613,48.224670,15.714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6986,0,0,0,1,8.589871,132.016100,4.296,1.088,0.765,4.80600,...,1.11,929,176.40,8.4,5638,4.296,1.088,298.74921,46.973351,14.478
6987,0,1,1,0,0.527699,131.705093,4.529,0.903,1.252,3.22210,...,29.35,2088,4500.53,453.3,5638,4.529,0.903,297.18875,47.093819,14.082
6988,0,0,0,0,1.739849,133.001270,4.444,1.031,0.043,3.11400,...,0.72,1608,1585.81,10.6,6119,4.444,1.031,286.50937,47.163219,14.757
6989,0,0,1,0,0.681402,132.181750,4.447,1.041,0.147,0.86500,...,1.07,2218,5713.41,12.3,6173,4.447,1.041,294.16489,47.176281,15.385


In [11]:
y = df[['koi_disposition']]
data_binary_encoded = pd.get_dummies(y, columns=["koi_disposition"])
data_binary_encoded.columns = [["candidate","confirmed","false_positive"]]

In [12]:
#from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(Xtemp, data_binary_encoded)

In [13]:
# # Random Forests in sklearn will automatically calculate feature importance
importances = rf.feature_importances_
importances

array([0.12696765, 0.10570641, 0.12729835, 0.04321933, 0.04140976,
       0.02788353, 0.01628051, 0.01693464, 0.03919883, 0.03072732,
       0.04721755, 0.07582893, 0.03014597, 0.03151336, 0.12482466,
       0.02135959, 0.0171224 , 0.01704446, 0.02081315, 0.01889476,
       0.01960885])

In [14]:
# We can sort the features by their importance
sorted(zip(rf.feature_importances_, Xtemp), reverse=True)

[(0.1272983488270639, 'koi_fpflag_co'),
 (0.12696764670071087, 'koi_fpflag_nt'),
 (0.12482466203931315, 'koi_model_snr'),
 (0.105706407513876, 'koi_fpflag_ss'),
 (0.0758289256488854, 'koi_prad'),
 (0.04721754983096429, 'koi_depth'),
 (0.04321932528158583, 'koi_fpflag_ec'),
 (0.04140976232480489, 'koi_period'),
 (0.03919882689470067, 'koi_impact'),
 (0.0315133611050037, 'koi_insol'),
 (0.030727317989119957, 'koi_duration'),
 (0.03014596895453725, 'koi_teq'),
 (0.027883532608376203, 'koi_time0bk'),
 (0.02135958969768092, 'koi_steff'),
 (0.020813149581988523, 'ra'),
 (0.019608854468539796, 'koi_kepmag'),
 (0.018894759357619546, 'dec'),
 (0.017122402268786707, 'koi_slogg'),
 (0.01704445886087579, 'koi_srad'),
 (0.016934643546788086, 'koi_srad'),
 (0.01628050649877855, 'koi_slogg')]

In [15]:
X = Xtemp.drop(columns=['ra','dec','koi_kepmag','koi_srad','koi_slogg'])
X

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_time0bk,koi_impact,koi_duration,koi_depth,koi_prad,koi_teq,koi_insol,koi_model_snr,koi_steff
0,0,0,0,0,54.418383,162.513840,0.586,4.50700,874.8,2.83,443,9.11,25.8,5455
1,0,1,0,0,19.899140,175.850252,0.969,1.78220,10829.0,14.60,638,39.30,76.3,5853
2,0,1,0,0,1.736952,170.307565,1.276,2.40641,8079.2,33.46,1395,891.96,505.6,5805
3,0,0,0,0,2.525592,171.595550,0.701,1.65450,603.3,2.75,1406,926.16,40.9,6031
4,0,0,0,0,4.134435,172.979370,0.762,3.14020,686.0,2.77,1160,427.65,40.2,6046
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6986,0,0,0,1,8.589871,132.016100,0.765,4.80600,87.7,1.11,929,176.40,8.4,5638
6987,0,1,1,0,0.527699,131.705093,1.252,3.22210,1579.2,29.35,2088,4500.53,453.3,5638
6988,0,0,0,0,1.739849,133.001270,0.043,3.11400,48.5,0.72,1608,1585.81,10.6,6119
6989,0,0,1,0,0.681402,132.181750,0.147,0.86500,103.6,1.07,2218,5713.41,12.3,6173


In [16]:
# Set features. This will also be used as your x values.
selected_features = X.columns
selected_features

Index(['koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co', 'koi_fpflag_ec',
       'koi_period', 'koi_time0bk', 'koi_impact', 'koi_duration', 'koi_depth',
       'koi_prad', 'koi_teq', 'koi_insol', 'koi_model_snr', 'koi_steff'],
      dtype='object')

In [17]:
#from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [18]:
print(X.shape, y.shape)

(6991, 14) (6991, 1)


In [19]:
X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_time0bk,koi_impact,koi_duration,koi_depth,koi_prad,koi_teq,koi_insol,koi_model_snr,koi_steff
6122,0,0,0,0,6.768901,133.077240,0.150,3.61600,123.1,1.24,1017,253.30,10.8,5737
6370,0,1,0,1,0.733726,132.020050,0.291,2.30900,114.6,0.86,1867,2891.64,13.8,5855
2879,1,0,0,0,7.652707,134.460380,0.970,79.89690,641.1,3.21,989,226.81,254.3,6328
107,0,0,0,0,7.953547,174.662240,0.300,2.63120,875.4,2.25,696,55.37,38.4,4768
29,0,0,0,0,4.959319,172.258529,0.831,2.22739,9802.0,12.21,1103,349.40,696.5,5712


In [20]:
# Scale your data
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [21]:
# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [22]:
# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [23]:
print(X_train_scaled.shape, y_train_categorical.shape)

(5243, 14) (5243, 3)


In [24]:
# create deep learning model
model = Sequential()
# give no. of columns in y_train_categorical as input_dim
model.add(Dense(units=100, activation='relu', input_dim=14))
model.add(Dense(units=100, activation='relu'))
# give of columns in y_train_categorical as units
model.add(Dense(units=3, activation='softmax'))

In [25]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [26]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               1500      
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 303       
Total params: 11,903
Trainable params: 11,903
Non-trainable params: 0
_________________________________________________________________


In [27]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=15,
    shuffle=True,
    verbose=2
)

Epoch 1/15
164/164 - 0s - loss: 0.5079 - accuracy: 0.7200
Epoch 2/15
164/164 - 0s - loss: 0.3784 - accuracy: 0.7847
Epoch 3/15
164/164 - 0s - loss: 0.3696 - accuracy: 0.7910
Epoch 4/15
164/164 - 0s - loss: 0.3642 - accuracy: 0.7934
Epoch 5/15
164/164 - 0s - loss: 0.3599 - accuracy: 0.8068
Epoch 6/15
164/164 - 0s - loss: 0.3605 - accuracy: 0.8016
Epoch 7/15
164/164 - 0s - loss: 0.3564 - accuracy: 0.8070
Epoch 8/15
164/164 - 0s - loss: 0.3553 - accuracy: 0.8100
Epoch 9/15
164/164 - 0s - loss: 0.3568 - accuracy: 0.8076
Epoch 10/15
164/164 - 0s - loss: 0.3535 - accuracy: 0.8064
Epoch 11/15
164/164 - 0s - loss: 0.3519 - accuracy: 0.8114
Epoch 12/15
164/164 - 0s - loss: 0.3524 - accuracy: 0.8117
Epoch 13/15
164/164 - 0s - loss: 0.3484 - accuracy: 0.8108
Epoch 14/15
164/164 - 0s - loss: 0.3467 - accuracy: 0.8161
Epoch 15/15
164/164 - 0s - loss: 0.3466 - accuracy: 0.8207


In [28]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(f"Deep Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

55/55 - 0s - loss: 0.3582 - accuracy: 0.8141
Deep Neural Network - Loss: 0.35817626118659973, Accuracy: 0.8140732049942017


In [29]:
 model.fit(
     X_train_scaled,
     y_train_categorical,
     epochs=80,
     shuffle=True,
     verbose=2
 )

Epoch 1/80
164/164 - 0s - loss: 0.3435 - accuracy: 0.8182
Epoch 2/80
164/164 - 0s - loss: 0.3429 - accuracy: 0.8159
Epoch 3/80
164/164 - 0s - loss: 0.3405 - accuracy: 0.8247
Epoch 4/80
164/164 - 0s - loss: 0.3366 - accuracy: 0.8207
Epoch 5/80
164/164 - 0s - loss: 0.3369 - accuracy: 0.8180
Epoch 6/80
164/164 - 0s - loss: 0.3334 - accuracy: 0.8255
Epoch 7/80
164/164 - 0s - loss: 0.3293 - accuracy: 0.8272
Epoch 8/80
164/164 - 0s - loss: 0.3299 - accuracy: 0.8287
Epoch 9/80
164/164 - 0s - loss: 0.3276 - accuracy: 0.8308
Epoch 10/80
164/164 - 0s - loss: 0.3265 - accuracy: 0.8335
Epoch 11/80
164/164 - 0s - loss: 0.3224 - accuracy: 0.8304
Epoch 12/80
164/164 - 0s - loss: 0.3232 - accuracy: 0.8306
Epoch 13/80
164/164 - 0s - loss: 0.3190 - accuracy: 0.8343
Epoch 14/80
164/164 - 0s - loss: 0.3190 - accuracy: 0.8344
Epoch 15/80
164/164 - 0s - loss: 0.3198 - accuracy: 0.8302
Epoch 16/80
164/164 - 0s - loss: 0.3171 - accuracy: 0.8333
Epoch 17/80
164/164 - 0s - loss: 0.3153 - accuracy: 0.8341
Epoch 

In [31]:
model_loss, model_accuracy = model.evaluate(
     X_test_scaled, y_test_categorical, verbose=2)
print(f"Deep Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

55/55 - 0s - loss: 0.2630 - accuracy: 0.8959
Deep Neural Network - Loss: 0.2630397081375122, Accuracy: 0.8958809971809387


In [32]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=100,
    shuffle=True,
    verbose=2
)

Epoch 1/100
164/164 - 0s - loss: 0.2509 - accuracy: 0.8877
Epoch 2/100
164/164 - 0s - loss: 0.2447 - accuracy: 0.8955
Epoch 3/100
164/164 - 0s - loss: 0.2450 - accuracy: 0.8913
Epoch 4/100
164/164 - 0s - loss: 0.2397 - accuracy: 0.8915
Epoch 5/100
164/164 - 0s - loss: 0.2427 - accuracy: 0.8953
Epoch 6/100
164/164 - 0s - loss: 0.2419 - accuracy: 0.8917
Epoch 7/100
164/164 - 0s - loss: 0.2399 - accuracy: 0.8945
Epoch 8/100
164/164 - 0s - loss: 0.2411 - accuracy: 0.8915
Epoch 9/100
164/164 - 0s - loss: 0.2449 - accuracy: 0.8915
Epoch 10/100
164/164 - 0s - loss: 0.2452 - accuracy: 0.8886
Epoch 11/100
164/164 - 0s - loss: 0.2368 - accuracy: 0.8970
Epoch 12/100
164/164 - 0s - loss: 0.2382 - accuracy: 0.8934
Epoch 13/100
164/164 - 0s - loss: 0.2357 - accuracy: 0.8964
Epoch 14/100
164/164 - 0s - loss: 0.2366 - accuracy: 0.8968
Epoch 15/100
164/164 - 0s - loss: 0.2368 - accuracy: 0.8951
Epoch 16/100
164/164 - 0s - loss: 0.2355 - accuracy: 0.8959
Epoch 17/100
164/164 - 0s - loss: 0.2370 - accura

In [33]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(f"Deep Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

55/55 - 0s - loss: 0.3580 - accuracy: 0.8862
Deep Neural Network - Loss: 0.35798099637031555, Accuracy: 0.8861556053161621


In [34]:
# make predictions and print the results
encoded_predictions = model.predict_classes(X_test_scaled[:5])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {y_test.values[:5].tolist()}")

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
Predicted classes: ['CONFIRMED' 'CANDIDATE' 'FALSE POSITIVE' 'FALSE POSITIVE'
 'FALSE POSITIVE']
Actual Labels: [['FALSE POSITIVE'], ['CANDIDATE'], ['FALSE POSITIVE'], ['FALSE POSITIVE'], ['FALSE POSITIVE']]


Deep Learning model with state 42 (V6) accuracy (88.61%) slightly better than random state 1 (V5) accuracy (88.55%).